In [11]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


# Import libraries 

In [12]:
import gromacs 

NOTE: Some configuration directories are not set up yet: 
	/Users/kansoali/.gromacswrapper
	/Users/kansoali/.gromacswrapper/qscripts
	/Users/kansoali/.gromacswrapper/templates
NOTE: You can create the configuration file and directories with:
	>>> import gromacs
	>>> gromacs.config.setup()


# Prepare the system

In [ ]:
!martinize2 -f model_start_aligned.pdb -ff martini3001 -x start_cg.pdb -o topoltemp.top -dssp /usr/bin/mkdssp -elastic -scfix -cys 0.40 -p backbone -pf 1000

In [ ]:
!python2.7 insane_altered.py -salt 0.15 -charge -0 -sol W -f start_cg.pdb -o system.gro -pbc rectangular -d 30 -x 9 -y 9 -z 12 -l POPC:100 -p topol.top

In [ ]:
!sed -i -e "s/Protein        1/molecule_0 1\nmolecule_1 1/g" ./topol.top

In [ ]:
!cat topol.top

# Energy Minimization

In [ ]:
#######6.0 double précision, pas trop utile#########

#!gmx grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c system.gro -r system.gro -p topol.top
!gmx_mpi grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c system.gro -r system.gro -p topol.top

In [ ]:
#!gmx mdrun -deffnm step6.0_minimization -gpu_id 0
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.0_minimization -ntomp 5 -v

In [ ]:
#!gmx grompp -f step6.1_minimization.mdp -o step6.1_minimization.tpr -c step6.0_minimization.gro -r step6.0_minimization.gro -p topol.top
!gmx_mpi grompp -f step6.1_minimization.mdp -o step6.1_minimization.tpr -c step6.0_minimization.gro -r step6.0_minimization.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.1_minimization -v
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.1_minimization -ntomp 5 -v

# Equilibration Steps - NVT

In [ ]:
#!gmx grompp -f step6.2_equilibration.mdp -o step6.2_equilibration.tpr -c step6.1_minimization.gro -r step6.1_minimization.gro -p topol.top
!gmx_mpi grompp -f step6.2_equilibration.mdp -o step6.2_equilibration.tpr -c step6.1_minimization.gro -r step6.1_minimization.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.2_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.2_equilibration -ntomp 5 -gpu_id 0 -v

# Equilibration Steps - NPT

In [ ]:
#!gmx grompp -f step6.3_equilibration.mdp -o step6.3_equilibration.tpr -c step6.2_equilibration.gro -r step6.2_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.3_equilibration.mdp -o step6.3_equilibration.tpr -c step6.2_equilibration.gro -r step6.2_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.3_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.3_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
#!gmx grompp -f step6.4_equilibration.mdp -o step6.4_equilibration.tpr -c step6.3_equilibration.gro -r step6.3_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.4_equilibration.mdp -o step6.4_equilibration.tpr -c step6.3_equilibration.gro -r step6.3_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.4_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.4_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
#!gmx grompp -f step6.5_equilibration.mdp -o step6.5_equilibration.tpr -c step6.4_equilibration.gro -r step6.4_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.5_equilibration.mdp -o step6.5_equilibration.tpr -c step6.4_equilibration.gro -r step6.4_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.5_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.5_equilibration -ntomp 5 -gpu_id 0 -v

In [ ]:
#!gmx grompp -f step6.6_equilibration.mdp -o step6.6_equilibration.tpr -c step6.5_equilibration.gro -r step6.5_equilibration.gro -p topol.top
!gmx_mpi grompp -f step6.6_equilibration.mdp -o step6.6_equilibration.tpr -c step6.5_equilibration.gro -r step6.5_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step6.6_equilibration -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step6.6_equilibration -ntomp 5 -gpu_id 0 -v

# MD run 

In [ ]:
#!gmx grompp -f step7_production.mdp -o step7_production.tpr -c step6.6_equilibration.gro -r step6.6_equilibration.gro -p topol.top
!gmx_mpi grompp -f step7_production.mdp -o step7_production.tpr -c step6.6_equilibration.gro -r step6.6_equilibration.gro -p topol.top

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step7_production -gpu_id 0 -v -nt 18
!gmx_mpi mdrun -rdd 3.0 -deffnm step7_production -ntomp 5 -gpu_id 0 -v

In [ ]:
!gmx check -f step7_production.xtc

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step7_production -cpi step7_production.cpt -gpu_id 0 -v -nt 18

# make an Index

In [ ]:
#get the number of the first and last atoms of the receptor and peptide
!cat system.gro

In [28]:
######################### In the terminal: (could'nt make it work here)######################################

!echo -e "a 1-725  \nname 22 rec \na 726-750 \nname 23 pep \nq" |gmx make_ndx -f system.gro -o index.ndx

#a 1-725
# name 22 rec
# a 726-750
# name 23 pep

                     :-) GROMACS - gmx make_ndx, 2022.2 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /Users/kansoali/Desktop/IBMM/simulations/cg_simu3
Command line:
  gmx make_ndx -f system.gro -o index.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:   304    Protein residues
There are:  5133      Other residues
There are:   108        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :  8686 atoms
  1 Protein             :   750 atoms
  2 Protein-H           :   750 atoms
  3 C-alpha             :     0 atoms
  4 Backbone            :     0 atoms
  5 MainChain           :     0 atoms
  6 MainChain+Cb        :     0 atoms
  7 MainChain+H         :     0 atoms
  8 SideChain           :   750 atoms
  9 SideChain-H         : 

# Center the the box

In [17]:
!echo 22 0|gmx trjconv -f step7_production.xtc -o step7_production.pbc.xtc -s step7_production.tpr -pbc mol -center -n index.ndx 

                     :-) GROMACS - gmx trjconv, 2022.2 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /Users/kansoali/Desktop/IBMM/simulations/cg_simu3
Command line:
  gmx trjconv -f step7_production.xtc -o step7_production.pbc.xtc -s step7_production.tpr -pbc mol -center -n index.ndx

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file step7_production.tpr, VERSION 2020.6 (single precision)
Reading file step7_production.tpr, VERSION 2020.6 (single precision)
Select group for centering
Group     0 (         System) has  8686 elements
Group     1 (        Protein) has   750 elements
Group     2 (      Protein-H) has   750 elements
Group     3 (        C-alpha) has     0 elements
Group     4 (       Backbone) has     0 elements
Group     5 (      MainChain) has     0 elements
Group     6 (   MainChain+Cb) has     0 elements
Group     7 (    

# NOW READY TO BE ANALYZED!!